### Sql queries

Prerequsites - sudo apt-get install python-mysqldb

This notebooks documents my attempt to replicate the analysis of Stats 19 data carried out by West Mids police as written up at https://trafficwmp.wordpress.com/2016/09/09/junction-malfunction-and-a-new-dawn/

This query gets the number of incidents involving a bicycle and somebody being killed or seriously injured.

In [16]:
'''
Example DbConfig.txt file

*************
[dbconfig]
host = localhost
user = <user>
userpw = <pass>
db = dft09-15
*************

'''

import MySQLdb
import ConfigParser
configParser = ConfigParser.ConfigParser()
configParser.readfp(open("DbConfig.txt"))
host = configParser.get("dbconfig", "host")
user = configParser.get("dbconfig", "user")
userpw = configParser.get("dbconfig", "userpw")
db = configParser.get("dbconfig", "db")

db = MySQLdb.connect(host = host, user = user, passwd = userpw, db = db)
cursor = db.cursor()

sql = ('''
       SELECT police_force.label, vehicle_type.vehicle_type, count(*) 
       FROM accidents a, vehicles v, police_force, vehicle_type, accident_severity
       WHERE a.accident_index=v.acc_index 
           AND a.police_force=police_force.code 
           AND year(a.date) in (2010,2011,2012,2013,2014) 
           AND a.accident_severity=accident_severity.code
           AND accident_severity.accident_severity in ("Fatal","Serious")
           AND v.vehicle_type = vehicle_type.code
           AND vehicle_type.vehicle_type = "Pedal Cycle"
           AND v.vehicle_type=vehicle_type.code 
           AND police_force.label="West Midlands";
''')

try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print row
except Exception as e:
    print "Error; query failed"
    print e

db.close()

('West Midlands', 'Pedal cycle', 530L)


This figure of 530 incidents correlates with WM's published figure.

In [18]:
'''
Example DbConfig.txt file

*************
[dbconfig]
host = localhost
user = <user>
userpw = <pass>
db = dft09-15
*************

'''

import MySQLdb
import ConfigParser
configParser = ConfigParser.ConfigParser()
configParser.readfp(open("DbConfig.txt"))
host = configParser.get("dbconfig", "host")
user = configParser.get("dbconfig", "user")
userpw = configParser.get("dbconfig", "userpw")
db = configParser.get("dbconfig", "db")

db = MySQLdb.connect(host = host, user = user, passwd = userpw, db = db)
cursor = db.cursor()

sql = ('''
SELECT count(*)
FROM accidents a, vehicles v, police_force, vehicle_type, accident_severity
WHERE a.accident_index = v.acc_index
AND v.vehicle_type = vehicle_type.code
AND vehicle_type.vehicle_type = "Pedal Cycle"
AND a.accident_severity=accident_severity.code
AND accident_severity.accident_severity in ("Fatal","Serious")
AND a.police_force = police_force.code
AND police_force.label
IN (
"West Midlands"
)
AND year( a.date )
IN ( 2010, 2011, 2012, 2013, 2014 )
and v.acc_index in (
SELECT v.acc_index
FROM vehicles v, vehicle_type
WHERE vehicle_type.vehicle_type <> "Pedal Cycle"
AND v.vehicle_type = vehicle_type.code
)
;
''')

try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        print row
except Exception as e:
    print "Error; query failed"
    print e

db.close()


(517L,)


517 incidents correlates.